In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.integrate import quad
from mpl_toolkits.mplot3d import Axes3D

const = pd.read_excel(r'C:\Users\steph\Downloads\test_data.xlsx')

k1_list = []
k2_list = []
k13_list = []
sigma_list = []
carb_list = []
G0_list = []
I0_list = []

Gpl_results = []
MGgut_results = []
Ipl_results = []
Irem_results = []
Usc1_results = []
Usc2_results = []

for i in range(51):
    k1_list.append(const.iloc[i][2])
    k2_list.append(const.iloc[i][3])
    k13_list.append(const.iloc[i][4])
    sigma_list.append(const.iloc[i][5])
    carb_list.append(const.iloc[i][6])
    G0_list.append(const.iloc[i][7])
    I0_list.append(const.iloc[i][8])


#def MGgut(t):
#    D_meal = inputs['Dmeal']
#    k1 = variables['k1']
#    sigma = variables['sigma']
#    return D_meal * np.exp(-(k1*t)**sigma) ######### error? This is Moutstomach... equation (2)

#Added by Stephen
def dMGgut_dt(t,MGgut):    
    # mGgut = mGmeal with a few edits in the most recent paper
    def mGgut(t):
        sigma = variables['sigma']
        k1 = variables['k1']
        Dmeal = inputs['Dmeal']
        k13 = variables['k13']
        return sigma*(k1**sigma)*(t**(sigma-1))*np.exp(-(k1*t)**sigma)*np.exp(-(k13*t))*Dmeal
    def mGpl(t,MGgut):
        k2 = variables['k2']
        return k2*MGgut   
    return mGgut(t) - mGpl(t,MGgut)


        
#Nathan's original with MGgut added where needed
def dGpl_dt(t, MGgut, Gpl, Ipl, Irem, Usc1, Usc2):
    def gliv(Gpl, Irem):
        gbliv = variables['gbliv']
        k3 = variables['k3']
        Gbpl = variables['Gbpl']
        k4 = variables['k4']
        beta = variables['beta'] 
        return gbliv - k3*(Gpl-Gbpl) - k4*beta*Irem

    def ggut(MGgut):
        k2 = variables['k2']
        f = variables['f']
        vG = variables['vG']
        Mb = inputs['Mb']
        return k2 * (f / (vG * Mb)) * MGgut

    def gnonit(Gpl):
        gbliv = variables['gbliv']
        KM = variables['K_M']
        Gbpl = variables['Gbpl']
        return gbliv * ((KM + Gbpl) / Gbpl) * (Gpl / (KM + Gpl))

    def git(Gpl, Irem):
        k5 = variables['k5']
        beta = variables['beta']
        KM = variables['K_M']
        return k5 * beta * Irem * (Gpl / (KM + Gpl))

    def gren(Gpl):
        c1 = variables['c1']
        vG = variables['vG']
        Mb = inputs['Mb']
        Gthpl = variables['Gthpl']
        return 0 if Gpl < Gthpl else (c1 / (vG * Mb)) * (Gpl - Gthpl)

    return gliv(Gpl, Irem) + ggut(MGgut) - gnonit(Gpl) - git(Gpl, Irem) - gren(Gpl)

def dUsc1_dt(t, Usc1):
        k10 = variables['k10']
        usa = inputs['usa']
        return usa(t) - k10 * Usc1

def dUsc2_dt(t, Usc1, Usc2):
        k9 = variables['k9']
        k10 = variables['k10']
        return k10 * Usc1 - k9 * Usc2

def dIpl_dt(t, MGgut, Gpl, Ipl, Irem, Usc1, Usc2):

    def ipnc(t, Gpl, dGpl_dt):
        Gbpl = variables['Gbpl']
        beta = variables['beta']
        k6 = variables['k6']
        k7 = variables['k7']
        tau_i = variables['tau_i']
        k8 = variables['k8']
        tau_d = variables['tau_d']
        t_start = variables['t_start']
        t_end = variables['t_end']

        if np.isscalar(dGpl_dt):
            dGpl_dt = np.array([dGpl_dt])  # Convert scalar to array

        def integrand(x):
            return np.squeeze(Gpl - Gbpl)

        integral_term, _ = quad(integrand, t_start, t_end)
        return (1 / beta) * (k6 * np.squeeze(Gpl - Gbpl) + (k7 / tau_i) * integral_term + (k7 / tau_i) * Gbpl + (k8 * tau_d) * dGpl_dt)


    def isa(t, Usc2):
        k9 = variables['k9']
        vI = variables['vI']
        Mb = inputs['Mb']
        return k9 * (1 / (vI * Mb)) * Usc2

    def ila(t, Ula):
        h = variables['h']
        t_half = variables['t_half']
        vI = variables['vI']
        Mb = inputs['Mb']
        return h * (t_half ** h) * (t ** (h - 1)) / (((t_half ** h) + (t ** h)) ** 2 * vI * Mb) * Ula(t)

    def iliv(t, Gpl, Ipl):
        k7 = variables['k7']
        beta = variables['beta']
        tau_i = variables['tau_i']
        Ibpl = variables['Ibpl']
        return k7 * (variables['Gbpl'] / (beta * tau_i * Ibpl)) * Ipl

    def irem(t, Ipl):
        k11 = variables['k11']
        Ibpl = variables['Ibpl']
        return k11 * (Ipl - Ibpl)

    return ipnc(t, Gpl, dGpl_dt(t, MGgut,Gpl, Ipl, Irem, Usc1, Usc2)) + isa(t, Usc2) + ila(t, inputs['Ula']) - iliv(t, Gpl, Ipl) - irem(t, Ipl)



def dIrem_dt(t, Gpl, Ipl, Irem, Usc1, Usc2):
    def ipl(t, Ipl):
        k11 = variables['k11']
        Ibpl = variables['Ibpl']
        return k11 * (Ipl - Ibpl)

    def iit(t, Irem):
        k12 = variables['k12']
        return k12 * Irem

    return ipl(t, Ipl) - iit(t, Irem)

                                                                              
def system(y, t):
    Gpl, MGgut, Ipl, Irem, Usc1, Usc2 = y

    # Define the derivatives
    dGpl_dt_val = dGpl_dt(t, MGgut, Gpl, Ipl, Irem, Usc1, Usc2)
    dMGgut_dt_val = dMGgut_dt(t,MGgut)
    dIpl_dt_val = dIpl_dt(t, MGgut, Gpl, Ipl, Irem, Usc1, Usc2)
    dIrem_dt_val = dIrem_dt(t, Gpl, Ipl, Irem, Usc1, Usc2)
    dUsc1_dt_val = dUsc1_dt(t, Usc1)
    dUsc2_dt_val = dUsc2_dt(t, Usc1, Usc2)

    return [dGpl_dt_val, dMGgut_dt_val, dIpl_dt_val, dIrem_dt_val, dUsc1_dt_val, dUsc2_dt_val]

for i in range(51):
    variables = {
        'k1': k1_list[i],
        'k2': k2_list[i],
        'k3': 0.13,
        'k4': 2.35e-4,
        'k5': 9.49e-2,
        'k6': 1.93e-1,
        'k7': 1.15,
        'k8': 7.27,
        'k9': 0,
        'k10': 0,
        'k11': 3.83e-2,
        'k12': 2.84e-1,
        'k13': k13_list[i],
        'sigma': sigma_list[i],
        'K_M': 13.2,
        'Gbpl': 5,
        'Ibpl': 5,
        'gbliv': 0.043,
        'Gthpl': 9,
        'vG': 17 / 70,
        'vI': 13 / 70,
        'beta': 1,
        'f': 0.005551,
        'tau_i': 31,
        't_start': 0,
        't_end': 30,
        't_start_ipnc': 0,
        't_end_ipnc': 15,
        'tau_d': 3,
        'c1': 0.1,
        'h': 1,
        't_half': 1,
        'a': 1,
        'b': 1
    }

    inputs = {
        'Dmeal': carb_list[i],
        'Mb': 66,
        'usa':  1,
        'Ula':  1,
        'h': 2
    }


    # Set the initial conditions
    initial_conditions = [G0_list[i],0, I0_list[i], 0, 0, 0]

    # Set the time points for integration
    t = np.linspace(0, 30, 100)

    # Solve the system of equations
    solution = odeint(system, initial_conditions, t)

    # Extract the variables from the solution
    Gpl_results.append(solution[:, 0])
    MGgut_results.append(solution[:, 1])
    Ipl_results.append(solution[:, 2])
    Irem_results.append(solution[:, 3])
    Usc1_results.append(solution[:, 4])
    Usc2_results.append(solution[:, 5])

plt.figure(figsize=(12, 12))  # Adjust the figure size as needed

# Plot Gpl
plt.subplot(3, 2, 1)
plt.yscale('log')
for i in range(51):
    plt.plot(t, Gpl_results[i], label='Gpl {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('Gpl')
plt.legend()

# Plot MGgut
plt.subplot(3, 2, 2)
plt.yscale('log')
for i in range(51):
    plt.plot(t, MGgut_results[i], label='MGgut {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('MGgut')
plt.legend()

# Plot Ipl
plt.subplot(3, 2, 3)
plt.yscale('log')
for i in range(51):
    plt.plot(t, Ipl_results[i], label='Ipl {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('Ipl')
plt.legend()

# Plot Irem
plt.subplot(3, 2, 4)
plt.yscale('log')
for i in range(51):
    plt.plot(t, Irem_results[i], label='Irem {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('Irem')
plt.legend()

# Plot Usc1
plt.subplot(3, 2, 5)
plt.yscale('log')
for i in range(51):
    plt.plot(t, Usc1_results[i], label='Usc1 {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('Usc1')
plt.legend()

# Plot Usc2
plt.subplot(3, 2, 6)
plt.yscale('log')
for i in range(51):
    plt.plot(t, Usc2_results[i], label='Usc2 {}'.format(i+1))
plt.xlabel('Time')
plt.ylabel('Usc2')
plt.legend()

plt.tight_layout()  # Adjust the spacing between subplots if needed
plt.show()

TypeError: 'int' object is not callable